# 작사가 인공지능 제작

## 데이터 전처리

### 데이터 읽어오기

In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/E04/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof', 'You saw her bathing on the roof', 'Her beauty and the moonlight overthrew her', 'She tied you', 'To a kitchen chair', 'She broke your throne, and she cut your hair', 'And from your lips she drew the Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah You say I took the name in vain', "I don't even know the name"]


### 데이터 전처리
* **모든 문자를 소문자처리**
* **각종 특수문자를 공백으로 치환하고, 공백을 하나의 공백으로 다시 치환해서 불필요한 문자열 제거**

##### 노드에서 제시한 데이터의 갯수가 도무지 맞춰지지 않아서 정제를 다시 해야한다고 판단하고 쉘에서 .txt파일들을 직접 검토해보았다. 노래 가사 그런지 중복이 워낙 많고(똑같은 가사 반복, 가사의 코러스도 모두 표기되어있음) 가사의 구성에 대한 표기를 해놓은 부분도 많았다. 가사와 코러스가 한줄에 함께 표기된 경우도 많았는데 하단의 정제를 거칠 경우 저 두 문장을 한 문장으로 인식하는 것 아닌가..? 하는 생각도 들었다.

![](./lyrics1.png)

![](lyrics2.png)

> 정보보안을 공부할 당시에 사용했던 사이트가 있다. 네트워크 패턴매칭 룰을 작성할 때도 정규표현식을 사용했기 때문에 가볍게 접한 적이 있었어서 좋은 결과를 살짝 기대하고 시도했는데, 정제된 데이터의 갯수가 말도 안되게 적거나 코드가 아예 안돌거나, 결과로 나온 문장이 매우 처참해서 그냥 엎어버렸다. 표기법이 조금 다른건지, 단순히 내가 실수를 한건지는 아직은 모르겠다. 나중에 NLP를 다시 접할 때 자세히 공부할 기회가 있겠지.<br> https://regexr.com/

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


**결국 그냥 노드에서 제시된 내용으로 진행했다.**

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]
len(corpus)

175749

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f9b0fa5e850>


In [5]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


In [18]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)


print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


**데이터의 갯수가 못내 거슬렸지만 어쩔 수가 없었다. 문장이라고 생각되지 않는 걸 만들어내는게 더욱 꼴보기 싫었기 때문에... 학습시간이 너무 오래걸리고 학업이 많이 밀려 긴 시간을 투자할 수 없는 것이 너무 아쉬웠다. 그래도 데이터의 갯수가 적은 것보다는 넘치는 것이 낫다고 생각해서 많을 때로 진행했다.**

In [7]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
dataset_val

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [9]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.6433188e-04,  7.5640084e-05,  1.8596736e-04, ...,
         -2.9185538e-05,  1.6883963e-04,  2.8372300e-04],
        [ 2.6321312e-04,  2.7067828e-05,  4.0277597e-04, ...,
         -2.0389973e-04,  4.0636532e-04,  7.7226450e-04],
        [ 3.0931630e-04,  1.1304940e-04,  4.9644895e-04, ...,
         -3.2776472e-04,  6.7705597e-04,  8.5512106e-04],
        ...,
        [ 5.2212918e-04,  1.9270877e-03, -1.0685378e-03, ...,
          1.1805564e-03, -7.3930726e-04,  9.7537108e-05],
        [ 5.7848648e-04,  2.2390580e-03, -1.2111373e-03, ...,
          1.3538847e-03, -9.7586075e-04, -7.9825331e-06],
        [ 6.3590478e-04,  2.5115414e-03, -1.3366953e-03, ...,
          1.4602967e-03, -1.1725180e-03, -1.4046836e-04]],

       [[ 1.6433188e-04,  7.5640084e-05,  1.8596736e-04, ...,
         -2.9185538e-05,  1.6883963e-04,  2.8372300e-04],
        [ 1.2539040e-04,  3.6243655e-04,  4.2163194e-04, ...,
          1.5040746e-04, 

In [10]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30, validation_data=dataset_val)

Epoch 1/30
549/549 [==============================] - 101s 181ms/step - loss: 3.7072 - val_loss: 3.3110
Epoch 2/30
549/549 [==============================] - 100s 182ms/step - loss: 3.2015 - val_loss: 3.1082
Epoch 3/30
549/549 [==============================] - 100s 182ms/step - loss: 3.0147 - val_loss: 2.9857
Epoch 4/30
549/549 [==============================] - 100s 182ms/step - loss: 2.8716 - val_loss: 2.8927
Epoch 5/30
549/549 [==============================] - 100s 182ms/step - loss: 2.7489 - val_loss: 2.8225
Epoch 6/30
549/549 [==============================] - 100s 181ms/step - loss: 2.6376 - val_loss: 2.7679
Epoch 7/30
549/549 [==============================] - 100s 181ms/step - loss: 2.5342 - val_loss: 2.7149
Epoch 8/30
549/549 [==============================] - 100s 181ms/step - loss: 2.4362 - val_loss: 2.6754
Epoch 9/30
549/549 [==============================] - 100s 181ms/step - loss: 2.3423 - val_loss: 2.6413
Epoch 10/30
549/549 [==============================] - 100s 182m

In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you lie <end> '

##### 다 제쳐놓고 완성된 문장이 제일 마음에 드는 것으로 저장함. 무려 에미넴의 노래 제목을 만들어냈다...

![](./song.jfif)

# 회고

### 처음으로 접한(?) NLP
**AI를 공부한 이래로(매우 아주 지극히 짧은 기간이지만) 처음으로 접한 자연어 처리 모델이다. AI를 시작하게 된 계기 자체가 자동차에 대한 관심과 자동차보안에 대한 흥미로 시작되었기 때문에 '나는 CV 외길이다.'라고만 생각했는데 꼭 그렇지도 않을 수 있겠다. 하긴 요즘에는 차량에도 'Hey Mercedes!' 같은 음성인식 서비스가 들어갈 뿐더러, 아직 내가 알지 못하는 곳에서 차량에도 접목되어 사용되고 있을 수 있겠다는 생각도 들긴 한다. 아무튼 결론은 생각보다 재미있었다는 것이다.<br>
<br>데이터의 측면에서는 아직도 이해하지 못하고 있는 '픽셀을 압축하여 블라블라블라~'보다는 '문장을 단어 단위로 나누어서 매칭되는 숫자 데이터 배열로 바꿔 저장하고 학습한다.' 라는 것이 나에겐 좀더 와닿았다. 마치 암호화과정의 그것과 비슷한 모습이랄까.(물론 내 수준의 이해 안에서만이다.) 물론 우리의 소리를 해석하는 과정에서는 파형이 어쩌고 하는 글을 얼핏 스친 적이 있긴한데, 일단은 거기까진 생각하지말자. ㅎㅎ**

### 총체적 난국
**이와 별개로 또 한번 큰 벽을 맞이하게 된 것도 있다. 별로 경험은 없지만 그래도 일반적인 코드를 작성할 때는 뭔가 에러가 표시되거나, 출력되는 에러가 없더라도 코드를 조금씩 잘라서 테스트하거나 등등 많은 방법으로 내가 옳게 가고 있는 것이 맞는지 확인할 수 있었다. 물론 그 것 또한 쉽지 않았지만 이 노드를 통해 인공지능에 있어서는 그것이 더욱 힘들다는 것을 알게 되었다.<br>
<br> 그래도 학습이 매우 금방 끝나는 그 동안은 커널이 다운될 정도로 미친듯한 광클로 돌려대고 결과를 여러번 비교하는 방식만으로도 스스로 납득할 수 있을 정도의 테스트는 충분했던 것 같다. 근데 이번 과제는 학습에 엄청나게 많은 시간이 들어갈 뿐더러, 정제된 데이터의 수나 출력된 loss등의 결과에도 불구하고 아주 평범한 문장이 출력되는 경우도 있었다. 이런 와중에 모델을 테스트해볼 다른 방법을 알지 못하는 나로써는 깨작깨작 파라미터나 데이터 정제 부분을 변경해보며 세 가지가 모두 충족될 때까지 다시 학습시키는 것만을 반복했다. 엄청나게 많은 시간을 잡아먹고 성과는 전혀 없으니(성과가 있는지 없는지 구분조차 불가능하다.) 매우 지쳐버렸다.<br>
<br>
아마 모델을 학습시키고 테스트하거나 문제점을 찾아내는 어떤 방법론들이 있지 않을까. 이 문제에 대해서도 조금 찾아보고 공부하는 시간을 가져야 할 듯 하다.**

### 지극히 외람되고 개인적인
**자고로 인공지능은 "인간"의 학습 방식을 컴퓨터에게 적용하는 것이라 했다. 좀 더 디테일하게 컴퓨터에게 인간의 "뇌"를 선물하는 것이 최종 목표가 아니겠는가. 미국의 저명한 언어학자이자 철학자인 'Avram Noam Chomsky'는 인간이 언어를 학습하는 방식이 인간의 자아 형성과 지적 성장, 나아가 인간 본성과 관계하며, 그로써 언어학과 철학이 맞닿아 있더랬다. 내 수많은 흥미 중 하나인 '인문학', 그 중에서도 '철학'이 어쩌면 AI와도 연결되어 있을 지 모르겠다. 먼 훗날 AI 고수가 된다면 NLP를 오히려 더 재미있게 할 수도 있지 않을까. 참 AI는 접목되지 않는 곳이 없는 것 같다. 나로써는 호재가 틀림없다. 그런 의미에서 'AI는 인문학을 먹고 산다.' 라는 책을 샀다. 사실 이거 읽을 시간에 코드 한줄을 더 써봐야 하는게 맞는데, 천성이 짧은 집중력이라 참 큰일이다.**